---
title: "MLE and COnjoint Analysis"
author: "Komal Nagaraj"
date: today
---



This assignment explores two methods for estimating the MNL model: (1) via Maximum Likelihood, and (2) via a Bayesian approach using a Metropolis-Hastings MCMC algorithm. 


## 1. Likelihood for the Multi-nomial Logit (MNL) Model

Suppose we have $i=1,\ldots,n$ consumers who each select exactly one product $j$ from a set of $J$ products. The outcome variable is the identity of the product chosen $y_i \in \{1, \ldots, J\}$ or equivalently a vector of $J-1$ zeros and $1$ one, where the $1$ indicates the selected product. For example, if the third product was chosen out of 3 products, then either $y=3$ or $y=(0,0,1)$ depending on how we want to represent it. Suppose also that we have a vector of data on each product $x_j$ (eg, brand, price, etc.). 

We model the consumer's decision as the selection of the product that provides the most utility, and we'll specify the utility function as a linear function of the product characteristics:

$$ U_{ij} = x_j'\beta + \epsilon_{ij} $$

where $\epsilon_{ij}$ is an i.i.d. extreme value error term. 

The choice of the i.i.d. extreme value error term leads to a closed-form expression for the probability that consumer $i$ chooses product $j$:

$$ \mathbb{P}_i(j) = \frac{e^{x_j'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

For example, if there are 3 products, the probability that consumer $i$ chooses product 3 is:

$$ \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{e^{x_1'\beta} + e^{x_2'\beta} + e^{x_3'\beta}} $$

A clever way to write the individual likelihood function for consumer $i$ is the product of the $J$ probabilities, each raised to the power of an indicator variable ($\delta_{ij}$) that indicates the chosen product:

$$ L_i(\beta) = \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} = \mathbb{P}_i(1)^{\delta_{i1}} \times \ldots \times \mathbb{P}_i(J)^{\delta_{iJ}}$$

Notice that if the consumer selected product $j=3$, then $\delta_{i3}=1$ while $\delta_{i1}=\delta_{i2}=0$ and the likelihood is:

$$ L_i(\beta) = \mathbb{P}_i(1)^0 \times \mathbb{P}_i(2)^0 \times \mathbb{P}_i(3)^1 = \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{\sum_{k=1}^3e^{x_k'\beta}} $$

The joint likelihood (across all consumers) is the product of the $n$ individual likelihoods:

$$ L_n(\beta) = \prod_{i=1}^n L_i(\beta) = \prod_{i=1}^n \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} $$

And the joint log-likelihood function is:

$$ \ell_n(\beta) = \sum_{i=1}^n \sum_{j=1}^J \delta_{ij} \log(\mathbb{P}_i(j)) $$



## 2. Simulate Conjoint Data

We will simulate data from a conjoint experiment about video content streaming services. We elect to simulate 100 respondents, each completing 10 choice tasks, where they choose from three alternatives per task. For simplicity, there is not a "no choice" option; each simulated respondent must select one of the 3 alternatives. 

Each alternative is a hypothetical streaming offer consistent of three attributes: (1) brand is either Netflix, Amazon Prime, or Hulu; (2) ads can either be part of the experience, or it can be ad-free, and (3) price per month ranges from \$4 to \$32 in increments of \$4.

The part-worths (ie, preference weights or beta parameters) for the attribute levels will be 1.0 for Netflix, 0.5 for Amazon Prime (with 0 for Hulu as the reference brand); -0.8 for included adverstisements (0 for ad-free); and -0.1*price so that utility to consumer $i$ for hypothethical streaming service $j$ is 

$$
u_{ij} = (1 \times Netflix_j) + (0.5 \times Prime_j) + (-0.8*Ads_j) - 0.1\times Price_j + \varepsilon_{ij}
$$

where the variables are binary indicators and $\varepsilon$ is Type 1 Extreme Value (ie, Gumble) distributed.

The following code provides the simulation of the conjoint data.

:::: {.callout-note collapse="true"}

In [ ]:
import numpy as np
import pandas as pd

# Set seed for reproducibility
np.random.seed(123)

# Define attributes
brands = ["N", "P", "H"]  # Netflix, Prime, Hulu
ads = ["Yes", "No"]
prices = list(range(8, 33, 4))

# Generate all possible profiles
profiles = pd.DataFrame(
    [(b, a, p) for b in brands for a in ads for p in prices],
    columns=["brand", "ad", "price"]
)
m = len(profiles)

# Assign part-worth utilities
b_util = {"N": 1.0, "P": 0.5, "H": 0.0}
a_util = {"Yes": -0.8, "No": 0.0}
p_util = lambda p: -0.1 * p

# Parameters
n_peeps = 100
n_tasks = 10
n_alts = 3

# Simulate one respondent's data
def sim_one(resp_id):
    tasks = []

    for t in range(1, n_tasks + 1):
        # Sample 3 alternatives randomly
        sampled = profiles.sample(n=n_alts, replace=False).copy()
        sampled["resp"] = resp_id
        sampled["task"] = t

        # Compute deterministic utility
        sampled["v"] = sampled["brand"].map(b_util) + \
                       sampled["ad"].map(a_util) + \
                       sampled["price"].apply(p_util)
        
        # Add Gumbel noise (Type I extreme value)
        sampled["e"] = -np.log(-np.log(np.random.uniform(size=n_alts)))
        sampled["u"] = sampled["v"] + sampled["e"]

        # Determine choice
        sampled["choice"] = (sampled["u"] == sampled["u"].max()).astype(int)

        tasks.append(sampled[["resp", "task", "brand", "ad", "price", "choice"]])

    return pd.concat(tasks)

# Simulate for all respondents
conjoint_data = pd.concat([sim_one(i) for i in range(1, n_peeps + 1)], ignore_index=True)
conjoint_data

::::



## 3. Preparing the Data for Estimation

The "hard part" of the MNL likelihood function is organizing the data, as we need to keep track of 3 dimensions (consumer $i$, covariate $k$, and product $j$) instead of the typical 2 dimensions for cross-sectional regression models (consumer $i$ and covariate $k$). The fact that each task for each respondent has the same number of alternatives (3) helps.  In addition, we need to convert the categorical variables for brand and ads into binary variables.


In [ ]:
df = pd.get_dummies(conjoint_data, columns=['brand', 'ad'], drop_first=True)
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)
df.head()

## 4. Estimation via Maximum Likelihood

Next we define the maximum likelihood function to find the  $\beta$ values for each parameter.


In [ ]:
X_cols = ['price', 'brand_N', 'brand_P', 'ad_Yes']
X = df[X_cols].values
y = df['choice'].values

n_alts = 3
n_sets = int(len(df) / n_alts)

def negative_log_likelihood(beta):
    ll = 0
    for i in range(n_sets):
        start = i * n_alts
        end = start + n_alts
        X_i = X[start:end]
        y_i = y[start:end]
        utilities = X_i @ beta
        utilities -= np.max(utilities)
        exp_utilities = np.exp(utilities)
        probabilities = exp_utilities / np.sum(exp_utilities)
        ll += np.sum(y_i * np.log(probabilities + 1e-12))
    return -ll

Further, we use the scipy.optimize() function to find the MLEs for the 4 parameters as well as  their standard errors (from the Hessian). We also estimate a 95% confidence interval for each parameter estimate.


In [ ]:
print("MLE Estimation of beta")
from scipy.optimize import minimize
import scipy.stats as stats
# Initial guess for parameters (zeros)
initial_beta = np.zeros(X.shape[1])

# Estimate MLE using BFGS method
result = minimize(negative_log_likelihood, initial_beta, method='BFGS')

# Estimated parameters
beta_hat = result.x

# Inverse of Hessian gives variance-covariance matrix
hessian_inv = result.hess_inv
standard_errors = np.sqrt(np.diag(hessian_inv))

# 95% Confidence Intervals
z = stats.norm.ppf(0.975)  # ≈ 1.96
conf_intervals = [(b - z*se, b + z*se) for b, se in zip(beta_hat, standard_errors)]
mle_results={}
# Display results
for name, b, se, ci in zip(X_cols, beta_hat, standard_errors, conf_intervals):
    print(f"{name}: beta = {b:.4f}, SE = {se:.4f}, 95% CI = ({ci[0]:.4f}, {ci[1]:.4f})")
    mle_results[name] = {
        "beta": b,
        "se": se,
        "ci": (ci[0], ci[1])
    }

## 5. Estimation via Bayesian Methods
In this section we are estimating the posterior distribution of 4 parameters in a MNL model using Bayesian inference. In other words we try to estimate "Based on the observed choices (data), and what I believed about the parameters before (priors), what should I now believe (posterior)"

The Baye's rule is given by - 

$$
P(\beta \mid \text{data}) \propto {P(\text{data} \mid \beta) \cdot P(\beta)}
$$

where $P(\beta)$ = Prior
$P(\text{data} \mid \beta)$ = Likelihood
$P(\beta \mid \text{data})$ = Posterior


In [ ]:
n_alts = 3
n_sets = int(len(df) / n_alts)
n_params = X.shape[1]

def log_prior(beta):
    logp_price = -0.5 * (beta[0] / 1)**2
    logp_binaries = -0.5 * np.sum((beta[1:] / 5)**2)
    return logp_price + logp_binaries

def log_posterior(beta):
    return log_prior(beta) - negative_log_likelihood(beta) 

#Metropolis-Hastings MCMC Sampler
n_samples = 11000
burn_in = 1000
samples = np.zeros((n_samples, n_params))

# Initial guess
current_beta = np.zeros(n_params)
current_log_post = log_posterior(current_beta)

# Proposal SDs: binary vars = 0.05, price = 0.005
proposal_sds = np.array([0.005, 0.05, 0.05, 0.05])

for t in range(1, n_samples):
    proposal = current_beta + np.random.normal(0, proposal_sds)
    proposal_log_post = log_posterior(proposal)
    accept_prob = min(1, np.exp(proposal_log_post - current_log_post))

    if np.random.rand() < accept_prob:
        current_beta = proposal
        current_log_post = proposal_log_post

    samples[t] = current_beta

# Drop burn-in
posterior_samples = samples[burn_in:]
bayes_results={}
# Posterior summaries
for i, name in enumerate(X_cols):
    param_samples = posterior_samples[:, i]
    mean = np.mean(param_samples)
    std = np.std(param_samples)
    ci_low, ci_high = np.percentile(param_samples, [2.5, 97.5])
    print(f"{name}: Beta = {mean:.4f}, SD = {std:.4f}, 95% CI = ({ci_low:.4f}, {ci_high:.4f})")
    bayes_results[name] = {
        "beta": mean,
        "std": std,
        "ci": (ci_low, ci_high)
    }

Below is the trace plot of the algorithm, as well as the histogram of the posterior distribution for the price estimate.


In [ ]:
print(posterior_samples)
import numpy as np
import matplotlib.pyplot as plt

# Assuming price is at index 3
price_samples = posterior_samples[:, 0]

# Plot
plt.figure(figsize=(12, 4))

# Trace Plot
# Trace Plot
plt.plot(price_samples, color='blue', linewidth=0.7)
plt.title("Trace Plot: Price Coefficient")
plt.xlabel("Iteration")
plt.ylabel("β (price)")
plt.show()

# Posterior Histogram
plt.hist(price_samples, bins=40, color='skyblue', edgecolor='black', density=True)
plt.title("Posterior Distribution: Price Coefficient")
plt.xlabel("β (price)")
plt.ylabel("Density")
plt.show()

The trace plot for the price coefficient shows stable oscillations around a central value, indicating that the Markov Chain Monte Carlo (MCMC) sampler has converged and is mixing well. There is no upward or downward drift, suggesting the chain is sampling effectively from the posterior distribution.

The corresponding posterior distribution is approximately normal, centered around -0.10. This confirms a strong negative effect of price on product choice — as price increases, the probability of selection decreases. The tight, symmetric shape of the distribution reflects high certainty in this estimate.

The following table summarizes the estimates from MLE and Bayesian methods.


In [ ]:
#| echo: false
from IPython.display import Markdown

# Build table header
table_md = "| Parameter | MLE Estimate (SE) | 95% CI (MLE) | Bayes Mean (SD) | 95% Credible Interval |\n"
table_md += "|-----------|-------------------|--------------|------------------|------------------------|\n"

# Fill in rows dynamically
for param in ['price', 'brand_N', 'brand_P', 'ad_Yes']:
    mle = mle_results[param]
    bayes = bayes_results[param]
    
    row = (
        f"| **{param}** "
        f"| {mle['beta']:.4f} ({mle['se']:.4f}) "
        f"| ({mle['ci'][0]:.4f}, {mle['ci'][1]:.4f}) "
        f"| {bayes['beta']:.4f} ({bayes['std']:.4f}) "
        f"| ({bayes['ci'][0]:.4f}, {bayes['ci'][1]:.4f}) |\n"
    )
    table_md += row

# Display the Markdown-formatted table
Markdown(table_md)

## 6. Discussion

Interpretation of Parameter Estimates
If the data were not simulated, the parameter estimates would reflect real-world consumer preferences inferred from observed choices. In that case:

A finding like $ \beta_\text{Netflix} > \beta_\text{Prime} $ means that, on average, consumers derive more utility from choosing Netflix compared to Amazon Prime, holding all other attributes constant.

This could indicate that consumers perceive higher value or satisfaction from Netflix’s offering (e.g., content library, user experience).

It also implies that, in the utility function a higher coefficient on Netflix leads to a higher probability of being chosen.

Regarding price:

A negative $\beta_\text{price}$ is expected and intuitive. It means that, all else equal, an increase in price decreases utility, which lowers the likelihood of the product being chosen.

This reflects basic economic theory: consumers prefer lower-cost options when utility from other features is equal.

#### Multi-Level (Hierarchical) Model

In real-world conjoint analysis, consumer preferences are rarely homogeneous. The basic multinomial logit (MNL) model assumes that every individual shares the same set of preference parameters ($\beta$), which is a strong and often unrealistic assumption.

To better reflect real-world heterogeneity in preferences, we use a **multi-level** model. Here's how this model can be both simulated and estimated:

**1. Simulating Individual-Level Preferences**

Instead of one global $\beta$, we assume each respondent $i$ has their own parameter vector ${\beta}_i$, drawn from a common population distribution:

$$
{\beta}_i \sim \mathcal{N}({\mu}, {\Sigma})
$$

- ${\mu}$: population mean of preferences  
- ${\Sigma}$: covariance matrix capturing variability and correlations between parameters  

This framework captures variation in individual tastes.

---

**2. Simulating Choice Data**

Using each individual's ${\beta}_i$, simulate their choices for each task using the softmax choice probability:

$$
P_{ij} = \frac{\exp(\mathbf{X}_{ij}^\top {\beta}_i)}{\sum_k \exp(\mathbf{X}_{ik}^\top {\beta}_i)}
$$

Where:
- $j$ indexes alternatives
- $\mathbf{X}_{ij}$ are the attributes of alternative $j$
- $P_{ij}$ is the probability that individual $i$ chooses alternative $j$

---

**3. Estimating the Model: Hierarchical Bayes**

To estimate a hierarchical model, use Hierarchical Bayesian (HB) methods such as:

- Gibbs sampling  
- Hamiltonian Monte Carlo 
- MCMC within Gibbs (as in traditional HB packages)

These methods estimate:

- Individual-level coefficients $\beta_i$
- Population-level parameters $\mu$, $\Sigma$

This approach gives:
- Personalized preference estimates
- More realistic modeling of population behavior
- Better predictive performance

---
